In [26]:
data Exp = Num Int
         | Exp :+: Exp
         | Exp :/: Exp
         | Var String
         | Let String Int Exp

type Env a b = a -> Maybe b

type ExpEnv = Env String Int

insert :: Eq a => a -> b -> Env a b -> Env a b
insert k v env k' = if k == k' then Just v else env k'

In [60]:
eval :: Exp -> Maybe Int
eval (Num n)     = Just n
eval (Var i)     = Just 0
eval (e1 :+: e2) = (+) <$> eval e1 <*> eval e2
eval (e1 :/: e2) = eval e2 >>= \e2' -> case e2' of 
    0 -> Nothing
    _ -> div <$> eval e1 <*> Just e2'
eval (Let x n e) = Just 0

In [61]:
eval $ Let "x" 3 $ Var "x" :+: Num 3

Just 0

In [63]:
import Control.Monad.State

evalState :: Exp -> State ExpEnv (Maybe Int)
evalState (Num n)     = pure (Just n)
evalState (Var i)     = do
                        env <- get
                        return $ env i
evalState (e1 :+: e2) = do
                        res1 <- evalState e1
                        res2 <- evalState e2
                        return  $ (+) <$> res1 <*> res2
evalState (e1 :/: e2) = do
                        e' <- evalState e2
                        case e' of
                            Just 0 -> return Nothing
                            _ -> do
                                e1' <- evalState e1
                                return $ div <$> e1' <*> e'

evalState (Let x n e) = do
    env <- get
    let env' = insert x n env
    put env'
    evalState e

In [4]:
{-# LANGUAGE LambdaCase #-}

evalState :: Exp -> State ExpEnv (Maybe Int)
evalState (Num n)     = pure (Just n)
evalState (Var i)     = get >>= \env -> return $ env i
evalState (e1 :+: e2) = evalState e1 >>= \res1 -> evalState e2 >>= \res2 -> return  $ (+) <$> res1 <*> res2
evalState (e1 :/: e2) = evalState e2 >>= \case 
                                    Just 0 -> return Nothing
                                    e2' -> evalState e1 >>= \e1' -> return $ div <$> e1' <*> e2'

In [87]:
main = do
    let exp = Let "x" 42 (Var "x") :+: Var "x"
    let (result, finalState) = runState (evalState exp) $ const Nothing
    putStrLn $ "Ergebnis: " ++ show result

main

Ergebnis: Just 84